In [ ]:
# import numpy and pandas, and DataFrame / Series
import math
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import scipy.optimize as optimize
import scipy
# Set some numpy options
np.set_printoptions(suppress=True)

# Set some pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# And some items for matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
#pd.options.display.mpl_style = 'default'

# read http://scikit-image.org/docs/dev/user_guide.html 
# for documentation
from skimage import io
from skimage import color

In [ ]:
labels = pd.read_csv("kaggle/trainLabels.csv")
m  = labels["ID"].max()
print (m)


X = None
img_size = 32
for i in np.arange(1, m+1):
    img = color.rgb2gray (io.imread("kaggle/train/{0}.Bmp".format(i)))
    img32 = scipy.misc.imresize(img, (img_size,img_size))
    img32[img32<128] = 0
    img32[img32>=128] = 1
    img_array = np.array(img32).flatten()
    img_array = img_array.reshape(1, img_array.shape[0])
    if(X is None):
        X = img_array
    else:
        X = np.vstack((X, img_array))
    
    #X = np.concatenate(([X,img32.flatten(1)]), axis=0)
    
print(X.shape)

In [ ]:
labels

In [ ]:
labels_s = labels["Class"].unique()
np.ndarray.sort(labels_s)
num_labels = labels_s.shape[0]
print(labels_s.shape[0])

In [ ]:

y = np.ones((m,1))*-1
for i in np.arange(labels_s.shape[0]):
    have_true = labels["Class"]==labels_s[i]
    for j in np.arange(m):
        if(have_true[j] == True):
            y[j] = i

In [ ]:
def displayData(X, example_width=None):
#DISPLAYDATA Display 2D data in a nice grid
#   [h, display_array] = DISPLAYDATA(X, example_width) displays 2D data
#   stored in X in a nice grid. It returns the figure handle h and the 
#   displayed array if requested.

    # Set example_width automatically if not passed in
    if example_width == None:
        example_width = round(math.sqrt(X.shape[1]));

    # Compute rows, cols
    m = X.shape[0]
    n = X.shape[1]
    example_height = round(n / example_width)
    
    # Compute number of items to display
    display_rows = math.floor(math.sqrt(m));
    display_cols = math.ceil(m / display_rows);

    # Between images padding
    pad = 1;
    
    # Setup blank display
    display_array = - np.ones((pad + (display_rows * (example_height + pad)), pad + (display_cols * (example_width + pad))))

    # Copy each example into a patch on the display array
    curr_ex = 0;
    for j in np.arange(display_rows):
        for i in np.arange(display_cols):
            if curr_ex > m : 
                break; 
            # Copy the patch

            # Get the max value of the patch
            max_val = np.amax(abs(X[curr_ex, :]));
            display_array_startx = j * (example_width + pad) #pad + (j * (example_height + pad)) - 1
            display_array_endx = ((j + 1) * (example_width + pad)) -1 # pad + (j * (example_height + pad))
            display_array_starty = i * (example_height + pad) # pad + (i * (example_width + pad))
            display_array_endy = ((i +1) * (example_height + pad)) - 1
                    
            de = (np.array(X[curr_ex, :]).reshape((example_width, example_height))) / max_val
            #print de.shape
            display_array[display_array_startx:display_array_endx, display_array_starty:display_array_endy] = (np.array(X[curr_ex, :]).reshape((example_width, example_height))) / max_val;
            curr_ex = curr_ex + 1;
        
        if curr_ex > m:
            break; 


    # Display Image
    plt.imshow(display_array, extent = [0,100,0,100], aspect='auto', cmap=plt.get_cmap('gray'))
    plt.show

In [ ]:
#% Randomly select 100 data points to display
rand_indices=np.random.permutation(m)
sel = X[rand_indices[0:100], :]
print(sel.shape)

In [ ]:
displayData(sel);

In [ ]:
def sigmoid(z):
#SIGMOID Compute sigmoid functoon
#   J = SIGMOID(z) computes the sigmoid of z.
    
    g = 1.0 / (1.0 + np.exp(-z));
    return g

In [ ]:
def lrCostFunction(theta_unraveled, X, y, lamda):
#LRCOSTFUNCTION Compute cost for logistic regression with 
#regularization
#   J = LRCOSTFUNCTION(theta, X, y, lambda) computes the cost of using
#   theta as the parameter for regularized logistic regression and the
#   gradient of the cost w.r.t. to the parameters. 

# Initialize some useful values
    m = y.shape[0] # number of training examples

# You need to return the following variables correctly 
    J = 0

# ====================== YOUR CODE HERE ======================
# Instructions: Compute the cost of a particular choice of theta.
#               You should set J to the cost.
#               Compute the partial derivatives and set grad to the partial
#               derivatives of the cost w.r.t. each parameter in theta
#
# Hint: The computation of the cost function and gradients can be
#       efficiently vectorized. For example, consider the computation
#
#           sigmoid(X * theta)
#
#       Each row of the resulting matrix will contain the value of the
#       prediction for that example. You can make use of this to vectorize
#       the cost function and gradient computations. 
#
# Hint: When computing the gradient of the regularized cost function, 
#       there're many possible vectorized solutions, but one solution
#       looks like:
#           grad = (unregularized gradient for logistic regression)
#           temp = theta; 
#           temp(1) = 0;   % because we don't add anything for j = 0  
#           grad = grad + YOUR_CODE_HERE (using the temp variable)
#
    theta_u = theta_unraveled.copy()
    theta_u = theta_u.flatten()
    theta = np.reshape(theta_u, (theta_u.shape[0], 1))
    
    hypotesis = sigmoid(X.dot(theta))
    tmp = 1/m*np.sum(-(y*np.nan_to_num(np.log(hypotesis)))-(1-y)*np.nan_to_num(np.log(1-hypotesis)))
    J = tmp + (lamda/(2*m))*(theta[1:].T.dot(theta[1:]) ) 
    
    
    gradient_a = 1/m*(X.T.dot(hypotesis-y))
    gradient_b = (lamda/m)*(theta)

    gradient_b[0,0]=0
    grad = gradient_a + gradient_b;
# =============================================================
    grad = grad.flatten()
        
    return (J[0,0], grad)

In [ ]:
def oneVsAll(X, y, num_labels, lamda):
#ONEVSALL trains multiple logistic regression classifiers and returns all
#the classifiers in a matrix all_theta, where the i-th row of all_theta 
#corresponds to the classifier for label i
#   [all_theta] = ONEVSALL(X, y, num_labels, lambda) trains num_labels
#   logisitc regression classifiers and returns each of these classifiers
#   in a matrix all_theta, where the i-th row of all_theta corresponds 
#   to the classifier for label i

# Some useful variables
    m = X.shape[0]
    n = X.shape[1]

# You need to return the following variables correctly 
    all_theta = np.zeros((num_labels, n+1))

# Add ones to the X data matrix
    X = np.hstack((np.ones((m,1)), X))

# ====================== YOUR CODE HERE ======================
# Instructions: You should complete the following code to train num_labels
#               logistic regression classifiers with regularization
#               parameter lambda. 
#
# Hint: theta(:) will return a column vector.
#
# Hint: You can use y == c to obtain a vector of 1's and 0's that tell use 
#       whether the ground truth is true/false for this class.
#
# Note: For this assignment, we recommend using fmincg to optimize the cost
#       function. It is okay to use a for-loop (for c = 1:num_labels) to
#       loop over the different classes.
#
#       fmincg works similarly to fminunc, but is more efficient when we
#       are dealing with large number of parameters.
#
# Example Code for fmincg:
#
#     % Set Initial theta
#     initial_theta = zeros(n + 1, 1);
#     
#     % Set options for fminunc
#     options = optimset('GradObj', 'on', 'MaxIter', 50);
# 
#     % Run fmincg to obtain the optimal theta
#     % This function will return theta and the cost 
#     [theta] = ...
#         fmincg (@(t)(lrCostFunction(t, X, (y == c), lambda)), ...
#                 initial_theta, options);
#
#    for i in np.arange(num_labels):
    for i in np.arange(num_labels):
        print("{0} of {1}".format(i+1, num_labels))
        initial_theta = np.zeros((X.shape[1], 1))  
        options = {'maxiter': 1000}
        # Try 'CG', 'BFGS', 'L-BFGS-B'
        Result = optimize.minimize(fun = lrCostFunction, x0 = initial_theta, args = (X, y==i, lamda), method = 'L-BFGS-B',
                                 #jac = lrCostGradFunction, options=options)
                                 jac=True, options=options)

        optimal_theta = Result.x
        
        if Result.success != True:
           print(Result.message)
        
        all_theta[i,:] = optimal_theta.flatten()
    
    return all_theta

In [ ]:
def predictOneVsAll(all_theta, X):
#PREDICT Predict the label for a trained one-vs-all classifier. The labels 
#are in the range 1..K, where K = size(all_theta, 1). 
#  p = PREDICTONEVSALL(all_theta, X) will return a vector of predictions
#  for each example in the matrix X. Note that X contains the examples in
#  rows. all_theta is a matrix where the i-th row is a trained logistic
#  regression theta vector for the i-th class. You should set p to a vector
#  of values from 1..K (e.g., p = [1; 3; 1; 2] predicts classes 1, 3, 1, 2
#  for 4 examples) 

    m = X.shape[0]
    num_labels = all_theta.shape[0]

# You need to return the following variables correctly 
    p = np.zeros((X.shape[0], 1))

# Add ones to the X data matrix
    X = np.hstack((np.ones((m,1)), X))

# ====================== YOUR CODE HERE ======================
# Instructions: Complete the following code to make predictions using
#               your learned logistic regression parameters (one-vs-all).
#               You should set p to a vector of predictions (from 1 to
#               num_labels).
#
# Hint: This code can be done all vectorized using the max function.
#       In particular, the max function can also return the index of the 
#       max element, for more information see 'help max'. If your examples 
#       are in rows, then, you can use max(A, [], 2) to obtain the max 
#       for each row.
#       

    p = np.argmax(X.dot(all_theta.T), axis=1)
    p = p.reshape((m,1))


# =========================================================================

    return p

In [ ]:
print ('\nTraining One-vs-All Logistic Regression...\n')

lamda_vec = [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]
lamda_vec = [0.001, 0.003, 0.01, 0.03, 0.1]
lamda_vec = [0.001]

#lamda_vec= [0.1]
acc = 0
winning_theta = None
winning_lamda = None
winning_pred = None
for la in lamda_vec:
    print('Trying Lambda {0}\n'.format(la))
    all_theta = oneVsAll(X, y, num_labels, la)
    pred = predictOneVsAll(all_theta, X)
    acc_tmp = np.mean(pred == y) * 100
    print('\n>>>>>>\nFor Lambda {0} Training Set Accuracy: \n{1}\n<<<<<\n'.format(la,acc_tmp))
    if acc_tmp > acc:
        acc = acc_tmp
        winning_theta = all_theta.copy()
        winning_lamda = la
        winning_pred = pred.copy()



In [ ]:
print('\nBest Training Set Accuracy with Lambda {0}: \n{1}'.format(winning_lamda, acc))

## Write kaggle submission file

In [ ]:
import glob

files_a = None
for file in glob.glob("kaggle/test/*.Bmp"):
    n_s= file.split('/')[2].split('.')[0]
    n = int(n_s)
    if(files_a is None):
        files_a = np.array(n)
    else:
        files_a = np.vstack((files_a, n))
        
np.ndarray.sort(files_a)       
files_a = files_a.flatten().tolist()

In [ ]:
TEST = None
img_size = 32
for i in files_a:
    img = color.rgb2gray (io.imread("kaggle/test/{0}.Bmp".format(i)))
    img32 = scipy.misc.imresize(img, (img_size,img_size))
    img32[img32<128] = 0
    img32[img32>=128] = 1
    img_array = np.array(img32).flatten()
    img_array = img_array.reshape(1, img_array.shape[0])
    if(TEST is None):
        TEST = img_array
    else:
        TEST = np.vstack((TEST, img_array))

In [ ]:
test_pred = predictOneVsAll(winning_theta, TEST)
print(labels_s[test_pred[0][0]], files_a[0])

In [ ]:
d = {'Class' : test_pred.flatten(), 'ID' : files_a}

df = pd.DataFrame(d)


check0 = []

for i in files_a:
    check0.append(y[i-1]==df[df["ID"]==i].Class.values[0])

    #print(y[j]==df[df["ID"]==j+1])

print (np.mean(check0) * 100)

In [ ]:
#df.to_csv("kaggle/test/submission.csv",index=False, header=True, cols=["ID","Class"])
y_test = []

for i in files_a:
    y_test.append(labels_s[df[df["ID"]==i].Class.values[0]])
f = open("kaggle/test/submission.csv", 'w')

print(len(y_test))
j = 0
f.write("ID,Class\n")
for i in files_a:
    f.write("{0},{1}\n".format(i, y_test[j]))
    j = j+1
    
f.close()

f = open("kaggle/test/submission2.csv", 'w')
j = 0
f.write("ID,Class\n")
for i in files_a:
    f.write("{0},{1}\n".format(files_a[j], labels_s[test_pred[j][0]]))
    j = j+1
    
f.close()    

## Sanity check for individual file

In [ ]:
#delete
i = 9928

img = color.rgb2gray (io.imread("kaggle/test/{0}.Bmp".format(i)))
img32 = scipy.misc.imresize(img, (img_size,img_size))
img32[img32<128] = 0
img32[img32>=128] = 1
img_array = np.array(img32).flatten()
img_array = img_array.reshape(1, img_array.shape[0])

l_pred = predictOneVsAll(winning_theta, img_array)

print(l_pred)
print(labels_s)
labels_s[l_pred[0,0]]

In [ ]:
X.shape

## Compare to Scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(X, y.flatten())
y_sklearn_predictions = lr.predict(X)
acc_tmp = np.mean(y_sklearn_predictions == y.flatten()) * 100
print('\n>>>>>>\nFor sklearn Training Set Accuracy: \n{0}\n<<<<<\n'.format(acc_tmp))
y_sklearn_test_predictions = lr.predict(TEST)

In [ ]:
d = {'scikit' : labels_s[y_sklearn_test_predictions.flatten().astype(int)].flatten(), 
     'logistic': labels_s[test_pred.flatten()], 'ID' : files_a}

df2 = pd.DataFrame(d)
l_vs_s = np.mean(test_pred.flatten()-y_sklearn_test_predictions.flatten().astype(int)==0) * 100
print("Logistic Regression is same as scikit logistic regression %{0:.2f} of time".format(l_vs_s))


In [ ]:
print(df2)

In [ ]:
f = open("kaggle/test/submission2-sklearn.csv", 'w')
j = 0
f.write("ID,Class\n")
for i in files_a:
    f.write("{0},{1}\n".format(files_a[j], labels_s[y_sklearn_test_predictions.flatten().astype(int)[j]]))
    j = j+1
    
f.close()  